# 05-supervisor.ipynb

### 멀티 에이전트 패턴 중 하나
- 도메인이 여러개 섞여있고
- 각 도메인마다 도구가 많고 복잡함
- 하위 담당 에이전트와 사용자가 소통할 필요가 없음.
- 단순 도구만 활용할 경우에는 사용 X

In [1]:
from dotenv import load_dotenv

load_dotenv()


True

In [ ]:
# from langchain_openai import ChatOpenAI # 이게 예전에는 표준이었음.

from langchain.chat_models import init_chat_model

model = init_chat_model('gpt-4.1-mini')

In [ ]:
# model.invoke('지금의 삼성전자 주가는 얼마야?') << 이렇게도 사용가능

AIMessage(content='현재 삼성전자의 주가는 실시간으로 변동되기 때문에 정확한 가격을 알려드리기 어렵습니다. 최신 삼성전자 주가를 확인하시려면 네이버 금융, 다음 금융, 또는 증권사 앱과 같은 금융 정보 제공 사이트를 참고하시기 바랍니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 17, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_cde3adbea5', 'id': 'chatcmpl-DD1vkzqWBohrIvcfA3WXrQ2G758xc', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c9350-8d4f-78f2-92bd-81b2bcadc3a8-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 17, 'output_tokens': 59, 'total_tokens': 76, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [35]:
from langchain.tools import tool

@tool
def create_calendar_event(
    title: str,
    start_time: str,
    end_time: str,
    attendees: list[str], #email
    location: str = '', # 위치가 없을 경우, 빈 문자열을 기본값으로
):
    '''캘린더 이벤트 생성'''
    return f'이벤트 생성 완료. {title} - {start_time} ~ {end_time}. '

@tool
def send_email(
    to: list[str],
    subject: str,
    body: str,
    attendees: list[str], #email

):
    '''이메일 발송'''
    return f'이메일 발송 완료. {to} - {subject}'

@tool
def get_available_time_slot(
    attendees: list[str], #email,
    date: str,
    duration_minutes: int
):
    '''참가자들이 특정 날짜에 참여 가능한 시간 확인'''

    return ['09:00', '14:00', '16:00']

@tool
def get_employees(
    str
):
    '''직원들의 이메일 주소 가져오기'''
    return str

In [36]:
from langchain_community.utilities import SQLDatabase
import os

DB_URI = os.environ.get('DB_URI')

db = SQLDatabase.from_uri(DB_URI)

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x150861550>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x150861550>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x150861550>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

In [61]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_agent
from datetime import datetime

toolkit = SQLDatabaseToolkit(db=db, llm=model)
dialect = db.dialect

SQL_AGENT_PROMPT = f"""
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. 

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.

추가 지침:
1. 사용자가 말한 팀 이름이 DB에 정확히 없을 수 있습니다. 
   반드시 'LIKE %키워드%'를 사용하거나, 팀 목록 테이블을 먼저 전체 조회하여 가장 유사한 팀을 찾아내세요.
2. 예: '인사 팀' 요청 시 -> '인사', 'HR', '인사관리' 등을 모두 염두에 두어야 합니다.
3. 이메일 주소를 찾을 때는 직원(employees) 테이블과 팀(teams/departments) 테이블을 적절히 조인(JOIN)하세요.
"""


sql_agent = create_agent(
    model,
    tools=toolkit.get_tools(), 
    system_prompt=SQL_AGENT_PROMPT
)

In [47]:
from langchain.agents import create_agent
from datetime import datetime

CALENDAR_AGENT_PROMPT = (
    "You are a calendar scheduling assistant. "
    "Parse natural language scheduling requests (e.g., 'next Tuesday at 2pm') "
    "into proper ISO datetime formats. "
    "Use get_available_time_slots to check availability when needed. "
    "Use create_calendar_event to schedule events. "
    "Always confirm what was scheduled in your final response."
    f'"NOW: {datetime.now()}"'
)

calendar_agent = create_agent(
    model,
    tools=[create_calendar_event, get_available_time_slot],
    system_prompt=CALENDAR_AGENT_PROMPT
)

In [ ]:
query = '다음주 화요일 오후 2시에 1시간 동안 팀 미팅을 잡아줘.'

for step in calendar_agent.stream(
    {'messages': [{'role': 'user', 'content': query}]}
):
    for update in step.values():
        for message in update.get('messages', []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  get_available_time_slot (call_6Ill7v7kOyO5xLQQKIutk6DQ)
 Call ID: call_6Ill7v7kOyO5xLQQKIutk6DQ
  Args:
    attendees: ['team']
    date: 2026-03-03
    duration_minutes: 60
================================= Tool Message =================================
Name: get_available_time_slot

["09:00", "14:00", "16:00"]
================================== Ai Message ==================================
Tool Calls:
  create_calendar_event (call_fpbiDe10Ck9l7OOFciK3DUo7)
 Call ID: call_fpbiDe10Ck9l7OOFciK3DUo7
  Args:
    title: 팀 미팅
    start_time: 2026-03-03T14:00:00
    end_time: 2026-03-03T15:00:00
    attendees: ['team']
    location:
================================= Tool Message =================================
Name: create_calendar_event

이벤트 생성 완료. 팀 미팅 - 2026-03-03T14:00:00 ~ 2026-03-03T15:00:00. 
================================== Ai Message ==================================

다음주 화요일인 2026년 3

In [ ]:
EMAIL_AGENT_PROMPT = (
    "You are an email assistant. "
    "Compose professional emails based on natural language requests. "
    "Extract recipient information and craft appropriate subject lines and body text. "
    "Use send_email to send the message. "
    "Always confirm what was sent in your final response."
)

email_agent = create_agent(
    model,
    tools=[send_email],
    system_prompt=EMAIL_AGENT_PROMPT,
)

In [30]:
query = '디자인 팀한테 내일 오전 10시에 있는 디자인 리뷰 리마인더 보내줘. '

for step in email_agent.stream(
    {'messages': [{'role': 'user', 'content': query}]}
):
    for update in step.values():
        for message in update.get('messages', []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  send_email (call_3qykbKCJW8wYzsGtEFgBZlhq)
 Call ID: call_3qykbKCJW8wYzsGtEFgBZlhq
  Args:
    to: ['design_team@example.com']
    subject: 디자인 리뷰 리마인더 - 내일 오전 10시
    body: 안녕하세요 디자인 팀,

내일 오전 10시에 예정된 디자인 리뷰 미팅을 리마인드 해드립니다. 모두 참석 부탁드립니다.

감사합니다.
    attendees: ['design_team@example.com']
================================= Tool Message =================================
Name: send_email

이메일 발송 완료. ['design_team@example.com'] - 디자인 리뷰 리마인더 - 내일 오전 10시
================================== Ai Message ==================================

디자인 팀에게 내일 오전 10시에 있는 디자인 리뷰 리마인더 이메일을 보냈습니다.


In [ ]:
@tool
def schedule_event(request: str) -> str:
    """Schedule calendar events using natural language.

    Use this when the user wants to create, modify, or check calendar appointments.
    Handles date/time parsing, availability checking, and event creation.

    # request 부분은 agent가 작성해서 보냄.

    Input: Natural language scheduling request (e.g., 'meeting with design team
    next Tuesday at 2pm')
    """
    result = calendar_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text

@tool
def get_employees_emails(request: str) -> str:
    """SQL 쿼리를 이용해 필요한 이메일 주소를 받아옴. 
    부서명이나 팀 이름을 입력하면 해당 소속 직원들의 이메일 리스트를 반환함.
    """
    
    # 지시사항을 더 명확하게 보강하여 전달
    full_request = f"데이터베이스에서 '{request}' 소속 직원들의 이메일 주소를 모두 찾아줘. 결과는 이메일 주소만 나열해줘."

    result = sql_agent.invoke({
        "messages": [{"role": "user", "content": full_request}]
    })
    
    return result["messages"][-1].content 
@tool
def manage_email(request: str) -> str:
    """Send emails using natural language.

    Use this when the user wants to send notifications, reminders, or any email
    communication. Handles recipient extraction, subject generation, and email
    composition.

    Input: Natural language email request (e.g., 'send them a reminder about
    the meeting')
    """
    result = email_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text

In [59]:
SUPERVISOR_PROMPT = '''너는 매우 유능한 개인 비서야.
너는 캘린더 이벤트를 조정하고, 이메일을 보낼 수 있어. SQL을 이용해서 직원 정보에서 이메일 주소도 확인할 수 있어.
사용자의 요청을 분석하여, 적절한 도구를 사용하고, 결과를 종합해야 해.
요청이 여러가지 액션을 취해야 하면, 순서를 잘 짜서 각종 도구들을 여러번 호출해.
'''

supervisor_agent = create_agent(
    model,
    tools=[schedule_event, get_employees_emails, manage_email ],
    system_prompt=SUPERVISOR_PROMPT,
)

In [62]:
query = '다음주 화요일 오후2시부터 1시간동안 인사 팀 미팅 잡고, 메일 다 보내놔.'

for step in supervisor_agent.stream(
    {'messages': [{'role': 'user', 'content': query}]}
):
    for update in step.values():
        for message in update.get('messages', []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_dJtcFnGiwZUaTTJd2a9T8U5T)
 Call ID: call_dJtcFnGiwZUaTTJd2a9T8U5T
  Args:
    request: 인사 팀 미팅 다음주 화요일 오후 2시부터 1시간
================================= Tool Message =================================
Name: schedule_event

인사 팀 미팅이 2026년 3월 3일 화요일 오후 2시부터 3시까지 일정으로 예약되었습니다. 필요한 다른 일정이 있으신가요?
================================== Ai Message ==================================
Tool Calls:
  get_employees_emails (call_JmtcgQkx6IRnKK7E7iyZ15pQ)
 Call ID: call_JmtcgQkx6IRnKK7E7iyZ15pQ
  Args:
    request: 인사 팀
================================= Tool Message =================================
Name: get_employees_emails

'인사 팀'과 유사한 'HR' 팀 소속 직원들의 이메일 주소는 다음과 같습니다:
hr1@company.com
hr2@company.com
hr3@company.com
hr4@company.com
hr5@company.com
================================== Ai Message ==================================
Tool Calls:
  manage_email (call_D2zRNXrTsX00NkAryD9eoU5N)
 Call ID